## IMPORT DATA

In [2]:
import torch
import torch.nn.functional as F
from ogb.nodeproppred import Evaluator, PygNodePropPredDataset
from torch.nn import LayerNorm, Linear, ReLU
from tqdm import tqdm

from torch_geometric.loader import NeighborLoader

In [109]:
import os.path as osp
import sys

import matplotlib.pyplot as plt
import torch
from ogb.nodeproppred import Evaluator, PygNodePropPredDataset
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T

# Download and process data at './dataset/ogbg_molhiv/'
# ogbnproteins_dataset = PygNodePropPredDataset('ogbn-proteins', root='../data')
ogbnarxiv_dataset = PygNodePropPredDataset(name='ogbn-arxiv', root= '../data')

## Prepare DataLoader

In [115]:
## DataLoader For ogbnproteins datasett
import os
from torch_geometric.utils import scatter
from torch_geometric.loader import RandomNodeLoader
split_idx = ogbnarxiv_dataset.get_idx_split()
data = ogbnarxiv_dataset[0]

data.y = data.y.to(torch.float)


# Set split indices to masks.
for split in ['train', 'valid', 'test']:
    mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    mask[split_idx[split]] = True
    data[f'{split}_mask'] = mask



device = f'cuda:{args.device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)


data = data.to(device)
train_idx = split_idx['train'].to(device)

In [116]:
data.edge_index

tensor([[104447,  15858, 107156,  ...,  45118,  45118,  45118],
        [ 13091,  47283,  69161,  ..., 162473, 162537,  72717]])

In [117]:
(data.x[data.train_mask] == data.x[train_idx])

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [118]:
data.x[data.train_mask].shape,data.y[data.train_mask].shape, data.x.size(-1)

(torch.Size([90941, 128]), torch.Size([90941, 1]), 128)

## Prepare Models

### GCN MODEL

In [123]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from ogb.nodeproppred import Evaluator
from tqdm import tqdm

    
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x.log_softmax(dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(data.x.size(-1), 64, data.y.size(-1)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)
criterion = torch.nn.BCEWithLogitsLoss()
evaluator = Evaluator('ogbn-arxiv')

def train():
    
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)[train_idx]
    
    loss = criterion(out.squeeze(1), data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


@torch.no_grad()
def test():
    model.eval()

    out = model(data.x, data.edge_index)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc


# def train(epoch):
#     model.train()

   

#     total_loss = total_examples = 0
#     for data in train_loader:
#         optimizer.zero_grad()
#         data = data.to(device)
#         out = model(data.x, data.edge_index)
#         loss = criterion(out[data.train_mask], data.y[data.train_mask])
#         loss.backward()
#         optimizer.step()

#         total_loss += float(loss) * int(data.train_mask.sum())
#         total_examples += int(data.train_mask.sum())



#     return total_loss / total_examples


# @torch.no_grad()
# def test():
#     model.eval()

#     y_true = {'train': [], 'valid': [], 'test': []}
#     y_pred = {'train': [], 'valid': [], 'test': []}



#     for data in test_loader:
#         data = data.to(device)
#         out = model(data.x, data.edge_index)

#         for split in y_true.keys():
#             mask = data[f'{split}_mask']
#             y_true[split].append(data.y[mask].cpu())
#             y_pred[split].append(out[mask].cpu())



#     train_rocauc = evaluator.eval({
#         'y_true': torch.cat(y_true['train'], dim=0),
#         'y_pred': torch.cat(y_pred['train'], dim=0),
#     })['acc']

#     valid_rocauc = evaluator.eval({
#         'y_true': torch.cat(y_true['valid'], dim=0),
#         'y_pred': torch.cat(y_pred['valid'], dim=0),
#     })['acc']

#     test_rocauc = evaluator.eval({
#         'y_true': torch.cat(y_true['test'], dim=0),
#         'y_pred': torch.cat(y_pred['test'], dim=0),
#     })['acc']

#     return train_rocauc, valid_rocauc, test_rocauc


print(f'Training And Test ')
for epoch in tqdm(range(1, 100001)):

    loss = train()
    if epoch%100==0:
   
        train_ac, valid_ac, test_ac = test()
        print(f'Loss: {loss:.4f}, Train: {train_ac:.4f}, '
              f'Val: {train_ac:.4f}, Test: {train_ac:.4f}')

Training And Test 


  0%|          | 100/100000 [01:21<27:40:13,  1.00it/s]

Loss: 0.6931, Train: 0.0048, Val: 0.0048, Test: 0.0048


  0%|          | 200/100000 [03:02<29:19:07,  1.06s/it]

Loss: 0.6931, Train: 0.0048, Val: 0.0048, Test: 0.0048


  0%|          | 208/100000 [03:10<25:21:32,  1.09it/s]


KeyboardInterrupt: 

### Node2vec Model

In [124]:
import os.path as osp
import sys
from tqdm import tqdm

from torch_geometric.nn import Node2Vec
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Node2Vec(
    data.edge_index,
    embedding_dim=data.y.size(-1),
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=1.0,
    q=1.0,
    sparse=True,
).to(device)

num_workers = 4 if sys.platform == 'linux' else 0
loader = model.loader(batch_size=128, shuffle=True, num_workers=num_workers)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)



@torch.no_grad()
def test():
    model.eval()
    z = model()
    y_true = {'train': [], 'valid': [], 'test': []}
    y_pred = {'train': [], 'valid': [], 'test': []}

 
   
    for split in y_true.keys():
        mask = data[f'{split}_mask']
        y_true[split].append(data.y[mask].cpu())
        y_pred[split].append(z[mask].cpu())


    train_rocauc = evaluator.eval({
        'y_true': torch.cat(y_true['train'], dim=0),
        'y_pred': torch.cat(y_pred['train'], dim=0),
    })['acc']

    valid_rocauc = evaluator.eval({
        'y_true': torch.cat(y_true['valid'], dim=0),
        'y_pred': torch.cat(y_pred['valid'], dim=0),
    })['acc']

    test_rocauc = evaluator.eval({
        'y_true': torch.cat(y_true['test'], dim=0),
        'y_pred': torch.cat(y_pred['test'], dim=0),
    })['acc']

    return train_rocauc, valid_rocauc, test_rocauc




print(f'Training And Test ')
for epoch in tqdm(range(1, 21)):

    loss = train()
   
    train_rocauc, valid_rocauc, test_rocauc = test()
    print(f'Loss: {loss:.4f}, Train: {train_rocauc:.4f}, '
          f'Val: {valid_rocauc:.4f}, Test: {test_rocauc:.4f}')

Training And Test 


  5%|▌         | 1/20 [01:31<28:59, 91.53s/it]

Loss: 1.0542, Train: 0.0000, Val: 0.0000, Test: 0.0000


  5%|▌         | 1/20 [02:18<43:59, 138.93s/it]


KeyboardInterrupt: 

## MLP model

In [11]:
from sklearn.neural_network import MLPClassifier
import numpy as np

X_train, X_valid, X_test = data.x[splitted_idx['train']], data.x[splitted_idx['valid']], data.x[splitted_idx['test']]

y_train, y_valid, y_test = data.y[splitted_idx['train']], data.y[splitted_idx['valid']], data.y[splitted_idx['test']]
X_train = torch.concat((X_train,X_valid))
y_train = torch.concat((y_train,y_valid))
model = MLPClassifier()
model.fit(X_train, y_train)

C:\Users\sambe\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sambe\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier()

In [13]:
from sklearn.metrics import accuracy_score
ypred_train = model.predict(data.x[splitted_idx['train']])
ypred_valid = model.predict(X_valid)
ypred_test = model.predict(X_test)

In [14]:
score_train = accuracy_score(ypred_train, data.y[splitted_idx['train']])
score_valid = accuracy_score(ypred_valid, y_valid)
score_test = accuracy_score(ypred_test, y_test)
print(f"Train score {score_train} , Valid score {score_valid}, Test score {score_test}")

Train score 0.6027204451237616 , Valid score 0.6176381757777106, Test score 0.5542867724214555


## GraphSAGE model

In [102]:
from torch_geometric.nn import GraphSAGE
model = GraphSAGE(
    data.num_node_features,
    hidden_channels=data.y.size(-1),
    num_layers=2,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


# def train(epoch):
#     model.train()

   

#     total_loss = total_examples = 0
#     for data in train_loader:
#         optimizer.zero_grad()
#         data = data.to(device)
#         out = model(data.x, data.edge_index)
#         loss = criterion(out[data.train_mask], data.y[data.train_mask])
#         loss.backward()
#         optimizer.step()

#         total_loss += float(loss) * int(data.train_mask.sum())
#         total_examples += int(data.train_mask.sum())



#     return total_loss / total_examples

# @torch.no_grad()
# def test():
#     model.eval()

#     y_true = {'train': [], 'valid': [], 'test': []}
#     y_pred = {'train': [], 'valid': [], 'test': []}



#     for data in test_loader:
#         data = data.to(device)
#         out = model(data.x, data.edge_index)

#         for split in y_true.keys():
#             mask = data[f'{split}_mask']
#             y_true[split].append(data.y[mask].cpu())
#             y_pred[split].append(out[mask].cpu())



#     train_rocauc = evaluator.eval({
#         'y_true': torch.cat(y_true['train'], dim=0),
#         'y_pred': torch.cat(y_pred['train'], dim=0),
#     })['acc']

#     valid_rocauc = evaluator.eval({
#         'y_true': torch.cat(y_true['valid'], dim=0),
#         'y_pred': torch.cat(y_pred['valid'], dim=0),
#     })['acc']

#     test_rocauc = evaluator.eval({
#         'y_true': torch.cat(y_true['test'], dim=0),
#         'y_pred': torch.cat(y_pred['test'], dim=0),
#     })['acc']

#     return train_rocauc, valid_rocauc, test_rocauc


def train():
    
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    loss = F.nll_loss(out.squeeze(1), data.y.squeeze(1)[train_idx].type(torch.LongTensor))
#     loss = F.nll_loss(out.squeeze(1), data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


@torch.no_grad()
def test():
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc



print(f'Training And Test ')
for epoch in tqdm(range(1, 100001)):

    loss = train()
    if epoch%100==0:
   
        train_ac, valid_ac, test_ac = test()
        print(f'Loss: {loss:.4f}, Train: {train_ac:.4f}, '
              f'Val: {train_ac:.4f}, Test: {train_ac:.4f}')

Training And Test 


  0%|          | 100/100000 [00:28<5:40:18,  4.89it/s]

Loss: -1.6484, Train: 0.0048, Val: 0.0048, Test: 0.0048


  0%|          | 200/100000 [00:58<11:15:50,  2.46it/s]

Loss: -2.6484, Train: 0.0048, Val: 0.0048, Test: 0.0048


...

Loss: -10.6485, Train: 0.0048, Val: 0.0048, Test: 0.0048


  1%|          | 1101/100000 [05:16<5:36:44,  4.89it/s] 

Loss: -11.6486, Train: 0.0048, Val: 0.0048, Test: 0.0048


  1%|          | 1193/100000 [05:46<7:58:33,  3.44it/s] 


KeyboardInterrupt: 